In [ ]:
# Author : Martin Thoma 
# URL : https://martin-thoma.com/classify-mnist-with-pybrain/

from struct import unpack
import gzip
import numpy as np
from numpy import zeros, uint8, float32

def get_labeled_data(imagefile, labelfile):
    """Read input-vector (image) and target class (label, 0-9) and return
       it as list of tuples.
    """
    # Open the images with gzip in read binary mode
    images = gzip.open(imagefile, 'rb')
    labels = gzip.open(labelfile, 'rb')

    # Read the binary data

    # We have to get big endian unsigned int. So we need '>I'

    # Get metadata for images
    images.read(4)  # skip the magic_number
    number_of_images = images.read(4)
    number_of_images = unpack('>I', number_of_images)[0]
    rows = images.read(4)
    rows = unpack('>I', rows)[0]
    cols = images.read(4)
    cols = unpack('>I', cols)[0]

    # Get metadata for labels
    labels.read(4)  # skip the magic_number
    N = labels.read(4)
    N = unpack('>I', N)[0]

    if number_of_images != N:
        raise Exception('number of labels did not match the number of images')

    # Get the data
    x = zeros((N, rows, cols), dtype=float32)  # Initialize numpy array
    y = zeros((N, 1), dtype=uint8)  # Initialize numpy array
    for i in range(N):
        if i % 1000 == 0:
            print("i: %i" % i)
        for row in range(rows):
            for col in range(cols):
                tmp_pixel = images.read(1)  # Just a single byte
                tmp_pixel = unpack('>B', tmp_pixel)[0]
                x[i][row][col] = tmp_pixel
        tmp_label = labels.read(1)
        y[i] = unpack('>B', tmp_label)[0]
    return (x, y)

x_train, y_train = get_labeled_data("train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz")
x_test, y_test = get_labeled_data("t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz")



i: 0
i: 1000
i: 2000
i: 3000
i: 4000
i: 5000
i: 6000
i: 7000
i: 8000
i: 9000
i: 10000
i: 11000
i: 12000
i: 13000
i: 14000
i: 15000
i: 16000
i: 17000
i: 18000
i: 19000
i: 20000
i: 21000
i: 22000
i: 23000
i: 24000
i: 25000
i: 26000
i: 27000
i: 28000
i: 29000
i: 30000
i: 31000
i: 32000
i: 33000
i: 34000
i: 35000
i: 36000
i: 37000
i: 38000
i: 39000
i: 40000
i: 41000
i: 42000
i: 43000
i: 44000
i: 45000
i: 46000
i: 47000
i: 48000
i: 49000
i: 50000
i: 51000
i: 52000
i: 53000
i: 54000
i: 55000
i: 56000
i: 57000
i: 58000
i: 59000
i: 0
i: 1000
i: 2000
i: 3000
i: 4000
i: 5000
i: 6000
i: 7000
i: 8000
i: 9000


In [ ]:
from pylab import imshow, show, cm


def view_image(image, label=""):
    """View a single image."""
    print("Label: %s" % label)
    imshow(image, cmap=cm.gray)
    show()
    
view_image(x_train[0], y_train[0])

<class 'numpy.ndarray'>


In [ ]:
from scipy.stats import norm

def naiveBayes(train_set, train_labels, test_set):
    # return predicted labels of test set

    # index corresponding to each label 
    dict_train_index = {}
    # probability corresponding to each label
    probability = {}
    for i in range(0, 10):
        dict_train_index[i] = list(np.where(np.asarray(train_labels) == i)[0])
        probability[i] = len(dict_train_index[i])/60000
    # dataset corresponding to each label
    dict_train_set = {}
    for i in range(0, 10):
        dict_train_set[i] = [train_set[j] for j in dict_train_index[i]]  
 
    # mean and variance for images with 28*28 features
    dict_mean_var = {}
    for i in range(0, 10):
        dict_mean_var[i] = [[0]*28]*28
        for j in range(0, 28):
            for k in range(0, 28):
                feature_list = []
                for m in range(0, len(dict_train_set[i])):
                    feature_list.append(dict_train_set[i][m][j][k])
                feature_mean = np.mean(feature_list)
                feature_var = np.var(feature_list)
                dict_mean_var[i][j][k] = (feature_mean, feature_var)

    # compare the probability of test set for each label
    test_label = []
    for set in test_set:
        dict_compare = {}
        for i in range(0, 10):
            dict_compare[i] = np.log(probability[i])
            for j in range(0, 28):
                for k in range(0, 28):
                    if dict_mean_var[i][j][k][1] == 0:
                        dict_compare[i]+=0
                    else:
                        dict_compare[i]+=np.log(norm.pdf(set[j][k], dict_mean_var[i][j][k][0], np.sqrt(dict_mean_var[i][j][k][1])))
        test_label.append(max(dict_compare, key=dict_compare.get))
    return test_label


label_result = naiveBayes(x_train[0:1000], y_train[0:1000], x_test[0:100])
correct = 0
for i in range(0, len(label_result)):
    if label_result[i] == y_test[i][0]:
        correct += 1
accuracy = correct/len(label_result)
print(accuracy)
print(label_result)

